In [25]:
from flask import Flask, render_template, Response, redirect, url_for
import cv2
import mediapipe as mp
import pandas as pd
import datetime
import time

In [26]:
#점수 조정을 위한 함수 (수정가능)

def sigmoid(x):
    return 1 / (1 + np.exp(-x+1))

In [27]:
app = Flask(__name__)


streaming_done = False  # 상태 변수

def run_model(filepath, select_yoga):
    import os
    global model_score
    select_yoga=5
    #BPE 로 이동
    os.chdir('C:\\Users\\USER\\yogafire\\BPE')
    
    #BPE 안의 모델 파일 실행 / training 안에 방금 저장된 유저의 파일과 , rightyogajoint에 미리 저장해둔 요가 자세의 올바른 자세 joint 값과 비교 
    output = os.popen(f"python inference_pair.py -v1 training/{filepath}.csv -v2 rightyogajoint/{select_yoga}.csv").read()
    
    # 실행된 결과에서 score만 가져오기
    list1 = output.split('\n')
    score_str = list1[-2]
    # score 점수 조정
    model_score = ((round(sigmoid((float(score_str.split(' ')[-1])))*10)*10))
    print(model_score)
    return model_score


def generate_frames():

    # MediaPipe 초기화
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    # 랜드마크 선택
    selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))
    
    # 컬럼 이름 정의
    columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
              ['neck_x', 'neck_y', 'hip_x', 'hip_y']
    
    data = []
    
    global streaming_done
    cap = cv2.VideoCapture(0)  # 카메라 열기
    frame_idx = 0
    start_time = time.time()
    duration = 10  # 30초 동안 실행

    while cap.isOpened() and time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        frame_data = [frame_idx]

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for idx in selected_landmarks:
                landmark = landmarks[idx]
                frame_data.extend([landmark.x, landmark.y])

            # 목과 엉덩이 좌표 계산
            neck_x = (landmarks[11].x + landmarks[12].x) / 2
            neck_y = (landmarks[11].y + landmarks[12].y) / 2
            hip_x = (landmarks[23].x + landmarks[24].x) / 2
            hip_y = (landmarks[23].y + landmarks[24].y) / 2
            frame_data.extend([neck_x, neck_y, hip_x, hip_y])

        else:
            frame_data.extend([None, None] * len(selected_landmarks))
            frame_data.extend([None, None, None, None])

        data.append(frame_data)

        # 시각화
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = frame.shape

            # 점 그리기
            for idx in selected_landmarks:
                x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
                cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)  # 파란색 점

            # 선 그리기
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx in selected_landmarks and end_idx in selected_landmarks:
                    start_x, start_y = int(landmarks[start_idx].x * w), int(landmarks[start_idx].y * h)
                    end_x, end_y = int(landmarks[end_idx].x * w), int(landmarks[end_idx].y * h)
                    cv2.line(frame, (start_x, start_y), (end_x, end_y), (255, 255, 255), 2)  # 하얀색 선

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()
    pose.close()

    # CSV 저장
    user_id = 'id123'
    suffix = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
    output_csv_path = f'./BPE/training/{user_id}_{suffix}.csv'
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")

    run_model(filepath, select_yoga)
    

    streaming_done = True  # 스트리밍 종료 상태 설정


In [ ]:
@app.route('/video_feed')
def video_feed():
    global streaming_done
    streaming_done = False  # 상태 초기화
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/processing')
def processing():
    return render_template('processing.html')

@app.route('/check_status')
def check_status():
    global streaming_done
    return {"status": "done" if streaming_done else "processing"}

@app.route('/redirect_after_stream')
def redirect_after_stream():
    if streaming_done:
        return redirect(url_for('processing'))
    else:
        return redirect(url_for('index'))


@app.route('/final_result')
def final_result():
    # 점수 계산이 완료된 후 결과 페이지
    return render_template('result.html',model_score=30)  # 예시: 점수 95


@app.route('/result2_page')
def result2_page():
    return render_template('result2.html')

@app.route('/process_model', methods=['POST'])
def process_model():
    
    # 모델 실행
    if input_data:
        score = run_model(filepath, select_yoga)
    else:
        score = 0  # 데이터가 없는 경우 기본 점수 반환

    return {"status": "success", "score": score}

def simulate_score_calculation():
    score_calculated = True


if __name__ == "__main__":
    app.run(host='localhost', port=8097)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [10/Dec/2024 14:17:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:15] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2024 14:17:15] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:15] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 14:17:16] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:18] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:19] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:20] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:22] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:22] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:23] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:24] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:25] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 -

Joint 데이터를 ./BPE/training/id123_241210_141732.csv에 저장했습니다.


127.0.0.1 - - [10/Dec/2024 14:17:32] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:32] "GET /redirect_after_stream HTTP/1.1" 302 -
127.0.0.1 - - [10/Dec/2024 14:17:32] "GET /processing HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:33] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:33] "GET /final_result HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:42] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2024 14:17:42] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:42] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 14:17:43] "GET /redirect_after_stream HTTP/1.1" 302 -
127.0.0.1 - - [10/Dec/2024 14:17:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:17:43] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2024 14:17:43] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 14:17:44] "GET /check_sta

Joint 데이터를 ./BPE/training/id123_241210_141800.csv에 저장했습니다.


127.0.0.1 - - [10/Dec/2024 14:18:01] "GET /redirect_after_stream HTTP/1.1" 302 -
127.0.0.1 - - [10/Dec/2024 14:18:01] "GET /processing HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:01] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:01] "GET /final_result HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:15] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2024 14:18:15] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:16] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 14:18:16] "GET /redirect_after_stream HTTP/1.1" 302 -
127.0.0.1 - - [10/Dec/2024 14:18:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:16] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2024 14:18:16] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 14:18:17] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:18] "GET /check_sta

Joint 데이터를 ./BPE/training/id123_241210_141833.csv에 저장했습니다.


127.0.0.1 - - [10/Dec/2024 14:18:34] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:34] "GET /redirect_after_stream HTTP/1.1" 302 -
127.0.0.1 - - [10/Dec/2024 14:18:34] "GET /processing HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:34] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:35] "GET /final_result HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:44] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2024 14:18:45] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:45] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 14:18:45] "GET /redirect_after_stream HTTP/1.1" 302 -
127.0.0.1 - - [10/Dec/2024 14:18:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:18:45] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2024 14:18:45] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2024 14:18:46] "GET /check_sta

Joint 데이터를 ./BPE/training/id123_241210_141903.csv에 저장했습니다.


127.0.0.1 - - [10/Dec/2024 14:19:04] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:19:04] "GET /redirect_after_stream HTTP/1.1" 302 -
127.0.0.1 - - [10/Dec/2024 14:19:04] "GET /processing HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:19:05] "GET /check_status HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2024 14:19:05] "GET /final_result HTTP/1.1" 200 -


In [7]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
import pandas as pd
import datetime
import time

In [10]:

app = Flask(__name__)

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 랜드마크 선택
selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))

# 컬럼 이름 정의
columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
          ['neck_x', 'neck_y', 'hip_x', 'hip_y']

data = []


def generate_frames():
    cap = cv2.VideoCapture(0)  # 카메라 열기
    frame_idx = 0
    start_time = time.time()
    duration = 30  # 30초 동안 실행

    while cap.isOpened() and time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        frame_data = [frame_idx]

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for idx in selected_landmarks:
                landmark = landmarks[idx]
                frame_data.extend([landmark.x, landmark.y])

            # 목과 엉덩이 좌표 계산
            neck_x = (landmarks[11].x + landmarks[12].x) / 2
            neck_y = (landmarks[11].y + landmarks[12].y) / 2
            hip_x = (landmarks[23].x + landmarks[24].x) / 2
            hip_y = (landmarks[23].y + landmarks[24].y) / 2
            frame_data.extend([neck_x, neck_y, hip_x, hip_y])

        else:
            frame_data.extend([None, None] * len(selected_landmarks))
            frame_data.extend([None, None, None, None])

        data.append(frame_data)

        # 시각화
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = frame.shape

            # 점 그리기
            for idx in selected_landmarks:
                x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
                cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)  # 파란색 점

            # 선 그리기
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx in selected_landmarks and end_idx in selected_landmarks:
                    start_x, start_y = int(landmarks[start_idx].x * w), int(landmarks[start_idx].y * h)
                    end_x, end_y = int(landmarks[end_idx].x * w), int(landmarks[end_idx].y * h)
                    cv2.line(frame, (start_x, start_y), (end_x, end_y), (255, 255, 255), 2)  # 하얀색 선
        
        # 프레임 인코딩
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()
    pose.close()

    # CSV 저장
    user_id = 'id123'
    suffix = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
    output_csv_path = f'./BPE/training/{user_id}_{suffix}.csv'
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")

    # 블러 처리 및 텍스트 추가
    while True:
        # 블러 처리
        frame = cv2.GaussianBlur(frame, (99, 99), 30)

        # 텍스트 추가
        text = "AI가 점수를 매기는 중..."
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        color = (255, 255, 255)  # 흰색
        thickness = 3
        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_x = (frame.shape[1] - text_size[0]) // 2
        text_y = (frame.shape[0] + text_size[1]) // 2

        cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, thickness)

        # 프레임 인코딩
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [ ]:

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('ex-edit.html')

if __name__ == "__main__":
    app.run(host='localhost', port=8097)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2024 11:30:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2024 11:30:13] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 11:30:13] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:30:18] "GET /video_feed HTTP/1.1" 200 -
Error on request:
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\serving.py", line 370, in run_wsgi
    execute(self.server.app)
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\serving.py", line 333, in execute
    for data in application_iter:
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\wsgi.py", line 256, in __next__
    return self._next()
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\wrappers\response.py", line 32, in _iter_encoded
    for item in iterable:
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_14324\18

Joint 데이터를 ./BPE/training/id123_241209_113048.csv에 저장했습니다.


127.0.0.1 - - [09/Dec/2024 11:40:30] "GET /static/css/index10.css HTTP/1.1" 304 -
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\ipykernel\kernelbase.py", line 437, in dispatch_shell
    await result
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\ipykernel\ipkernel.py", line 362, in execute_request
    await super().execute_request(stream, ident, parent)
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\ipykernel\kernelbase.py", line 787, in execute_request
    time.sleep(self._execute_sleep)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\opencv\lib\logging\__init__.py", line 1086, in emit
    stream.write(msg + self.terminator)
OSError: [WinError 6] 핸들이 잘못되었습니다
Call stack:
  File "C:\Users\USER\anaconda3\envs\opencv\lib\runpy.py", line 197, in _run_module_as_main
    return _r

In [1]:
# from flask import Flask,render_template,request  ## flask 라이브러리에서 Flask import
# import pymysql
# import pandas as pd

In [10]:
# app = Flask(__name__)

# con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
#                       port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
#                       autocommit=True # 결과 DB 반영 (Insert or update)
#                      )
# cur = con.cursor()

# # @app.route('/boot/ex/<int:seqidx>/<user_id>')
# # def hello_world(seqidx,user_id):
# #     return render_template('eltag.html')

# # @app.route('/boot/ex/')
# # def hello_world(seqidx,user_id):
# #     return render_template('eltag.html')



# if __name__ == "__main__":
#     app.run(host='localhost', port=8097)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit


In [ ]:
# from flask import Flask, render_template, Response
# import cv2
# import mediapipe as mp
# import pandas as pd
# import datetime
# import time

# app = Flask(__name__)

# # MediaPipe 초기화
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# # 랜드마크 선택
# selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))

# # 컬럼 이름 정의
# columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
#           ['neck_x', 'neck_y', 'hip_x', 'hip_y']

# data = []

# def generate_frames():
#     cap = cv2.VideoCapture(0)  # 카메라 열기
#     frame_idx = 0
#     start_time = time.time()
#     duration = 30  # 30초 동안 실행

#     while cap.isOpened() and time.time() - start_time < duration:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         frame_idx += 1
#         rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         results = pose.process(rgb_frame)
#         frame_data = [frame_idx]

#         if results.pose_landmarks:
#             landmarks = results.pose_landmarks.landmark
#             for idx in selected_landmarks:
#                 landmark = landmarks[idx]
#                 frame_data.extend([landmark.x, landmark.y])

#             # 목과 엉덩이 좌표 계산
#             neck_x = (landmarks[11].x + landmarks[12].x) / 2
#             neck_y = (landmarks[11].y + landmarks[12].y) / 2
#             hip_x = (landmarks[23].x + landmarks[24].x) / 2
#             hip_y = (landmarks[23].y + landmarks[24].y) / 2
#             frame_data.extend([neck_x, neck_y, hip_x, hip_y])

#         else:
#             frame_data.extend([None, None] * len(selected_landmarks))
#             frame_data.extend([None, None, None, None])

#         data.append(frame_data)

#         # 시각화
#         if results.pose_landmarks:
#             for idx in selected_landmarks:
#                 x, y = int(landmarks[idx].x * frame.shape[1]), int(landmarks[idx].y * frame.shape[0])
#                 cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)

#         _, buffer = cv2.imencode('.jpg', frame)
#         frame = buffer.tobytes()

#         yield (b'--frame\r\n'
#                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

#     cap.release()
#     pose.close()

#     # CSV 저장
#     user_id = 'id123'
#     suffix = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
#     output_csv_path = f'./BPE/training/{user_id}_{suffix}.csv'
#     df = pd.DataFrame(data, columns=columns)
#     df.to_csv(output_csv_path, index=False)
#     print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")


# @app.route('/video_feed')
# def video_feed():
#     return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

# @app.route('/')
# def index():
#     return render_template('ex-edit.html')

# if __name__ == "__main__":
#     app.run(host='localhost', port=8097)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [08/Dec/2024 22:33:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2024 22:33:51] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2024 22:33:52] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Dec/2024 22:33:58] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2024 22:33:58] "GET /favicon.ico HTTP/1.1" 404 -
